In [13]:
from svpg.utils import load_algo
from svpg.algos.svpg import RBF

import torch
from torch.nn.utils import parameters_to_vector

In [17]:
action_agents, _, _, _ = load_algo("../runs/RllCartPoleSwingUp-v0/22-05-13/17-08-30/A2C-independent")

theta_i = action_agents[0].parameters()
theta_j = action_agents[1].parameters()

params = [
    parameters_to_vector(action_agent.parameters())
    for action_agent in action_agents
]
params = torch.stack(params)

kernel = RBF()(params, params.detach())